In [ ]:
# Top Products Analysis

  # Get top 10 products
query = """
SELECT 
    p.product_name,
    SUM(od.quantity) AS quantity,
    SUM(od.unit_price * od.quantity * (1 - od.discount)) AS revenue
FROM order_details od
JOIN products p ON od.product_id = p.product_id
GROUP BY p.product_name
ORDER BY revenue DESC
LIMIT 10;
"""
df = pd.read_sql(query, engine)

# Create dual-axis plot
fig = px.bar(df, x='product_name', y='quantity', 
             title='Top 10 Products: Quantity vs Revenue',
             labels={'quantity': 'Quantity Sold', 'product_name': 'Product'})

# Add revenue line
fig.add_scatter(x=df['product_name'], y=df['revenue'], 
                name='Revenue', mode='lines+markers', 
                yaxis='y2', line=dict(color='orange'))

# Update layout
fig.update_layout(
    yaxis2=dict(title='Revenue', overlaying='y', side='right'),
    xaxis_tickangle=-45,
    legend=dict(x=0.5, y=1.1, orientation='h')
)

fig.show()


In [ ]:
# Revenue-based ABC classification
  
   # Query to calculate product revenue 
query = """
SELECT 
    p.product_name,
    SUM(od.unit_price * od.quantity) AS revenue
FROM order_details od
JOIN products p ON od.product_id = p.product_id
GROUP BY p.product_name
ORDER BY revenue DESC;
"""
df = pd.read_sql(query, engine)

# ABC classification based on revenue
df = df.sort_values(by='revenue', ascending=False).reset_index(drop=True)
df['Cumulative revenue'] = df['revenue'].cumsum()
df['Cumulative %'] = 100 * df['Cumulative revenue'] / df['revenue'].sum()
df['Class'] = pd.cut(df['Cumulative %'], bins=[0, 80, 95, 100], labels=['A', 'B', 'C'])

fig, ax1 = plt.subplots(figsize=(12, 6))

# Bar chart
ax1.bar(df['product_name'], df['revenue'], color='royalblue')
ax1.set_ylabel('revenue', color='blue')
ax1.tick_params('y', colors='blue')
ax1.set_xticklabels(df['product_name'], rotation=45, ha='right',fontsize=8)

# Cumulative % line
ax2 = ax1.twinx()
ax2.plot(df['product_name'], df['Cumulative %'],  color='red', marker='o', markersize=4, linestyle='-', linewidth=2)
ax2.set_ylabel('Cumulative %', color='red')
ax2.tick_params('y', colors='red')
ax2.axhline(80, color='green', linestyle='--', label='80% (Class A limit)')
ax2.axhline(95, color='orange', linestyle='--', label='95% (Class B limit)')
ax2.legend(loc='lower right')

plt.title('ABC Classification of Products by Revenue ')
plt.tight_layout()
plt.show()

In [ ]:
# Discount Impact Analysis by Product Category

  # Query: Discount impact by category
query = """
SELECT 
    c.category_name,
    od.discount * 100 AS discount_percentage,
    SUM(od.quantity) AS total_quantity_sold,
    SUM(od.unit_price * od.quantity * (1 - od.discount)) AS total_revenue
FROM order_details od
JOIN products p ON od.product_id = p.product_id
JOIN categories c ON p.category_id = c.category_id
WHERE od.discount > 0
GROUP BY c.category_name, od.discount
ORDER BY c.category_name, od.discount;
"""
df = pd.read_sql(query, engine)

fig = px.line(
    df,
    x="discount_percentage", y="total_quantity_sold", color="category_name",
    facet_col="category_name", facet_col_wrap=4, markers=True,
    title="Discount Impact Analysis by Product Category",
    labels={"discount_percentage": "Discount (%)", "total_quantity_sold": "Quantity Sold"},
    hover_data=["total_revenue"], template="plotly_white",
)

fig.update_layout(
    height=650,
    showlegend=False,
    title_font=dict(size=20, family="Arial", color="#333"),
    hovermode="x unified",
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()